In [1]:
# import stuff
import pandas as p
import numpy as n
import seaborn as s

## Data Set 2; Stock Exchange Data

This is the data set containing the stock market information for 14 unique exchanges over the span of 12-31-1965 to 06-03-2021.

There are 3 csv files, but for this project, we will only be using indexData.csv. This has all the null values and extra data that indexProcessed.csv does not. 

[csv link](https://www.kaggle.com/mattiuzc/stock-exchange-data)


### Useful Stats:
- Size of set ~> 112,457
- Columns ~> 8

In [5]:
stock = p.read_csv('data/indexData.csv', header = None, skipinitialspace = True,
                encoding = 'latin-1', low_memory = False, skiprows=1,
                names = ['Index', 'Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
)
p.set_option('display.max_rows', 100)

In [3]:
stock

,Index,Date,Open,High,Low,Close,Adj Close,Volume
0,NYA,1965-12-31,528.690002,528.690002,528.690002,528.690002,528.690002,0.0
1,NYA,1966-01-03,527.210022,527.210022,527.210022,527.210022,527.210022,0.0
2,NYA,1966-01-04,527.840027,527.840027,527.840027,527.840027,527.840027,0.0
3,NYA,1966-01-05,531.119995,531.119995,531.119995,531.119995,531.119995,0.0
4,NYA,1966-01-06,532.070007,532.070007,532.070007,532.070007,532.070007,0.0
...,...,...,...,...,...,...,...,...
112452,N100,2021-05-27,1241.119995,1251.910034,1241.119995,1247.069946,1247.069946,379696400.0
112453,N100,2021-05-28,1249.469971,1259.209961,1249.030029,1256.599976,1256.599976,160773400.0
112454,N100,2021-05-31,1256.079956,1258.880005,1248.140015,1248.930054,1248.930054,91173700.0
112455,N100,2021-06-01,1254.609985,1265.660034,1254.609985,1258.579956,1258.579956,155179900.0


In [26]:
# To determine null values, and how many there are
stock.isna().sum()

Index           0
Date            0
Open         2204
High         2204
Low          2204
Close        2204
Adj Close    2204
Volume       2204
dtype: int64

A) The types of data being sampled are the values of 14 major stock indices from around the world. The tickers are listed below:

- NYA 
- IXIC
- HSI
- 000001.SS
- GSPTSE
- 399001.SZ
- NSEI
- GDAXI
- KS11
- SSMI
- TWII
- J203.JO
- N225
- N100

B) The features given by the data are the date, the open, close, and adjusted close. The high and low daily values as well as the volume are also given. These are general stats for all stock parket indices. 

C) 
### Qualatative & Nomial:
- Index

### Qualatative & Ordinal:
- Date

### Quantitative & Ordinal:
- Open
- High
- Low
- Close
- Adj Close
- Volume

All the values are discrete

D) There are 2204 total null values in every quantitative column. This is due to potential dates that stocks did not happen, rather than neglect. Dates such as holidays or a certain day the stock market was closed. 

E) These feautures are chosen because as stated earlier, they are typical generic stock attributes and are very good for data analysis.

F) Some other features may be how many hours of trading were in each day. Some days are cut short for certain reasons. Other than that, not sure what else directly relates to the indices themselves. Maybe leader of the country at that time?

In [81]:
# G) Doing a pivot on this data

# Figuring out which dates have null values
stockNull = stock.isnull()
stockNull = stockNull.any(axis = 1)
stockNulls = stock[stockNull]
stockNulls

,Index,Date,Open,High,Low,Close,Adj Close,Volume
289,NYA,1967-02-23,NaN,NaN,NaN,NaN,NaN,NaN
26639,HSI,1987-01-01,NaN,NaN,NaN,NaN,NaN,NaN
26659,HSI,1987-01-29,NaN,NaN,NaN,NaN,NaN,NaN
26660,HSI,1987-01-30,NaN,NaN,NaN,NaN,NaN,NaN
26706,HSI,1987-04-06,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
108316,N100,2005-03-28,NaN,NaN,NaN,NaN,NaN,NaN
108511,N100,2005-12-26,NaN,NaN,NaN,NaN,NaN,NaN
110051,N100,2012-01-02,NaN,NaN,NaN,NaN,NaN,NaN
110643,N100,2014-05-01,NaN,NaN,NaN,NaN,NaN,NaN


In [110]:
#sort the number of nulls
SNcount = p.DataFrame()
SNcount['Index'] = stockNulls[['Index']]
SNcount['Total Nulls'] = 0
SNcount = SNcount.groupby('Index').count()
SNcount = SNcount.sort_values('Total Nulls', ascending = False)

"""This is important because now we will know how many values will have NaN when we do our pivots."""
SNcount

,Total Nulls
Index,
N225,626
HSI,258
GSPTSE,250
000001.SS,172
399001.SZ,168
GDAXI,168
SSMI,159
KS11,152
TWII,141
